In [ ]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Now, there will be times when you'd like to do something in SQL that... well isn't possible in SQL. That's one of the reason we're covering _three_ tools in this course. An additional option, however, is to define what's known as a UDF, or "User Defined Function." Certain SQL dialects (Redshift, BigQuery, DuckDB) support UDFs in languages other than SQL.

If you end up working as an analyst or engineer, it's likely UDFs will come in handy at some point. This lesson is mainly to let you know they exist and show you some basic functionality. DuckDB supports Python UDFs— so if you'd like to use a function in SQL on a dataset, you can!

UDFs take a regular function and make them accessible to DuckDB using the `create_function`, function. In the next cell, we'll create a function that returns a random emoji per row. In DuckDB the `side_effects` parameter is to apply the function to every row when the result is independent of the row, like for random data.

In [ ]:
from duckdb.typing import *
from faker import Faker


def random_emoji():
    fake = Faker()
    fake.add_provider("emoji")
    return fake.emoji()


conn.create_function("random_emoji", random_emoji, [], "VARCHAR", side_effects=True)

Since we created the function in the connection, we can call it directly in SQL. This one is a bit playful:

In [ ]:
%%sql
SELECT
    fullname,
    random_emoji() as 👋
FROM nps_public_data.parks
LIMIT 10;
    

Now, that might not be the most _useful_ function, so let's give another example. Can you think of how to swap the casing of every letter in SQL? Me neither. Luckily, it's a pretty standard function in most libraries. Here, we use the pyarrow implementation to define a `swap_case` UDF

In [ ]:
import pyarrow.compute as pc


def swap_case(x):
    # Swap the case of the 'column' using utf8_swapcase and return the result
    return pc.utf8_swapcase(x)


# To register the function, we must define it's type to be 'arrow'
conn.create_function("swap_case", swap_case, ["VARCHAR"], "VARCHAR", type="arrow")

This let's up perform what might be an otherwise complex operation in SQL with relative simplicity _directly_ in the SQL context.

In [ ]:
%%sql
SELECT
    swap_case(fullname) as pARK_nAME
FROM nps_public_data.parks
LIMIT 10

DuckDB also supports _lambdas_, which operate on every item in a list. Take for example `list_transform`, which can [act on every element in a list](https://duckdb.org/docs/sql/functions/lambda.html#transform).

This is important: there are sometimes when exploding a list (as we did in earlier sections) would be a nightmare. Lambda functions are a great way to keep a table at a certain grain while being able to apply transformations across those lists.

In the following example, we apply a list transform to modify the entries of our `states_list`s.

In [ ]:
%%sql
-- Which parks are fully or partially in Utah?
WITH park_states AS (
    SELECT 
        fullname,
        states AS states_string, 
        split(states, ',') ::string[] AS states_list
    FROM nps_public_data.parks p
    )
SELECT 
    fullname,
    states_list,
    -- Transform lists of strings to lowercase
    list_transform(states_list, x -> lower(x)) as lower_states_list,
    -- You can even combine lambdas with UDFs
    list_transform(states_list, x -> swap_case(x)) as lower_states_list,
FROM park_states
WHERE list_contains(states_list, 'UT')
LIMIT 5

Other functions like `list_reduce` or `list_filter` can be helpful, too. Here's another example:

In [ ]:
%%sql
-- Which parks are fully or partially in Utah?
WITH park_states AS (
    SELECT 
        fullname,
        states AS states_string, 
        split(states, ',') ::string[] AS states_list
    FROM nps_public_data.parks p
    )
SELECT 
    fullname,
    states_string,
    states_list,
    -- Transform list back to a string 🙃
    list_reduce(states_list, (x, y) -> concat(x, ', ', y)) as new_states_string,
    -- Filter out utah from list
    list_filter(states_list, x -> x != 'UT') as states_list_no_ut,
FROM park_states
WHERE list_contains(states_list, 'UT')
AND len(states_list) > 1
LIMIT 5

Note how we were able to use `list_reduce` to transform our list _back_ to a string and `list_filter` to _remove_ an element from our list.

So our _patterns_ for this section are:

- **User Defined Functions (UDFs):** can be valuable when you need the flexibility of Python or JavaScript, but would prefer your function execute in SQL, whether for practical or organizational purposes. UDFs can be incredibly valuable, but you should keep a close eye on performance. They're often overlooked by query optimizers and can quickly lead to bottlenecks in performance. 
- **Lambdas:** can be used to operate on every element in a list. This can be valuable for keeping `STRUCT` columns intact without needing to explode and re-create them... A very computationally intensive operation. These functions may go by different names in different databases, or might not be possible. 